In [1]:
import os

import agama
import h5py
import matplotlib.pyplot as plt
import numpy as np
from gc_utils import iteration_name, main_prog_halt, snapshot_name  # type: ignore
from matplotlib.animation import PillowWriter

In [2]:
sim = "m12i"

sim_dir = "/Users/z5114326/Documents/simulations/"
data_dir = "/Users/z5114326/Documents/GitHub/gc_kinematics/data/"

fire_dir = sim_dir + sim + "/" + sim + "_res7100/"

proc_file = sim_dir + sim + "/" + sim + "_processed.hdf5"
proc_data = h5py.File(proc_file, "r")  # open processed data file

agama.setUnits(mass=1, length=1, velocity=1)

In [3]:
it = 0
grp_int = 8580896

In [4]:
it_id = iteration_name(it)
source_dat = proc_data[it_id]["source"]

source_mask = (np.array(source_dat["group_id"]) == grp_int) & (np.array(source_dat["analyse_flag"]) == 1)

snap_acc = np.unique(source_dat["snap_acc"][source_mask])[0]

In [9]:
proc_data[it_id]["snapshots"]["snap600"].keys()

<KeysViewHDF5 ['acc_snap', 'ek', 'ep_fire', 'et_norm', 'gc_id', 'group_id', 'lx', 'ly', 'lz', 'lz_norm', 'mass', 'phi_cyl', 'ptype', 'r', 'r_cyl', 'vphi_cyl', 'vr_cyl', 'vx', 'vy', 'vz', 'x', 'y', 'z']>

In [7]:
key_lst = []
for key in proc_data[it_id]["snapshots"].keys():
    if int(key[4:]) >= snap_acc:
        key_lst.append(key)

In [8]:
kin_dict = {}

for key in key_lst:
    snap_dict = proc_data[it_id]["snapshots"][key]
    snap_mask = np.array(snap_dict["group_id"]) == grp_int

    snap = int(key[4:])
    pot_file = data_dir + "potentials/" + sim + "/snap_%d/combined_snap_%d.ini" % (snap, snap)
    pot = agama.Potential(pot_file)
    af = agama.ActionFinder(pot, interp=False)

    et = np.array(snap_dict["et"][snap_mask])
    lz = np.array(snap_dict["lz"][snap_mask])

    r_circs = pot.Rcirc(E=et)
    xyz = np.column_stack((r_circs, r_circs * 0, r_circs * 0))
    v_circs = np.sqrt(-r_circs * pot.force(xyz)[:, 0])
    vel = np.column_stack((v_circs * 0, v_circs, v_circs * 0))
    init_conds = np.concatenate((xyz, vel), axis=1)
    lz_circ = af(init_conds)[:, 2]

    E_0 = pot.potential((0, 0, 0))

    lz_norm = lz / np.array(lz_circ)
    et_norm = et / np.abs(E_0)

    kin_dict[key] = {}

    kin_dict[key]["lz"] = lz
    kin_dict[key]["et"] = et

    kin_dict[key]["lz_norm"] = lz_norm
    kin_dict[key]["et_norm"] = et_norm

In [19]:
np.unique(np.array(source_dat["group_id"]))

array([      -2,       -1,        0,   265300,   707419,   835961,
        1157444,  1297671,  1356451,  1574823,  1674369,  1920378,
        1920380,  2184477,  2422680,  2800535,  3113442,  3159643,
        3739370,  3994037,  4361373,  4414957,  4906476,  5242313,
        5583343,  6808839,  8580896,  9106256,  9454779, 13687078,
       14194861, 15431197, 16199669, 19898495, 21324265])

In [10]:
proc_data.close()